In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
import random
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from collections import defaultdict

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed,
)


In [ ]:
def seed_everything(seed=42):
    """Stabilize Python/NumPy/Torch randomness plus Transformers seed."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    set_seed(seed)

seed_everything(42)

In [4]:
TRAIN_JSON_PATH = "/content/drive/MyDrive/nlp/train.json"
DEV_JSON_PATH   = "/content/drive/MyDrive/nlp/dev.json"
SYN_JSON_PATH   = "/content/drive/MyDrive/nlp/merged_updated.json"
MODEL_NAME = "microsoft/deberta-v3-large"

In [ ]:
def load_json_records(path, key=None):
    """Load JSON annotations and optionally unwrap a nested key."""
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if key:
        data = data[key]
    return list(data.values()) if isinstance(data, dict) else data

In [ ]:
def build_examples(records):
    """Flatten raw JSON structures into plain dicts Trainer can digest."""
    # Normalize: flatten nested lists so every item is a dict
    def flatten(xs):
        for x in xs:
            if isinstance(x, list):
                yield from flatten(x)
            else:
                yield x

    # If a single dict is passed
    if isinstance(records, dict):
        records = [records]

    # Flatten any nested structure
    records = list(flatten(records))

    out = []

    for r in records:
        if not isinstance(r, dict):
            continue

        pre = (r.get("precontext") or "").strip()
        sent = (r.get("sentence") or "").strip()
        end = (r.get("ending") or "").strip()
        meaning = (r.get("judged_meaning") or "").strip()

        meaning_txt = (
            "Meaning (intended definition):\n"
            f"{meaning}"
        )

        # Build a readable story block to pair with the definition prompt.
        story_parts = [pre, sent]
        if end:
            story_parts.append(end)

        story_txt = "Story:\n" + "\n".join(story_parts)

        ex_sent = (r.get("example_sentence") or "").strip()
        if ex_sent:
            story_txt += f"\nExample sentence: {ex_sent}"

        label = float(r.get("average", 0.0))

        gid = f"{r.get('homonym', '')}||{sent}"

        # Carry extra metadata to help downstream error analysis.
        out.append({
            "context": story_txt,
            "meaning": meaning_txt,
            "label": label,
            "stdev": float(r.get("stdev", 0)),
            "group_id": gid,
            "choices": r.get("choices"),
            "nonsensical": r.get("nonsensical"),
            "sample_id": r.get("sample_id")
        })

    return out


In [ ]:
# Load each split and convert into flat DataFrames for quick inspection.
train_records = load_json_records(TRAIN_JSON_PATH)
dev_records   = load_json_records(DEV_JSON_PATH)
syn_records   = load_json_records(SYN_JSON_PATH)
train_df = pd.DataFrame(build_examples(train_records))
dev_df   = pd.DataFrame(build_examples(dev_records))
syn_df   = pd.DataFrame(build_examples(syn_records))
print("TRAIN:", len(train_df))
print("DEV:", len(dev_df))
print("SYN:", len(syn_df))


TRAIN: 2280
DEV: 588
SYN: 792


In [8]:
print(train_df.head())

                                             context  \
0  Story:\nThe old machine hummed in the corner o...   
1  Story:\nThe old machine hummed in the corner o...   
2  Story:\nThe old machine hummed in the corner o...   
3  Story:\nThe old machine hummed in the corner o...   
4  Story:\nThe old machine hummed in the corner o...   

                                             meaning  label     stdev  \
0  Meaning (intended definition):\nthe difference...    3.0  1.581139   
1  Meaning (intended definition):\nthe inherent c...    3.8  0.836660   
2  Meaning (intended definition):\nthe difference...    2.2  1.303840   
3  Meaning (intended definition):\nthe inherent c...    4.4  0.894427   
4  Meaning (intended definition):\nthe difference...    2.6  1.516575   

                                         group_id          choices  \
0  potential||The potential couldn't be measured.  [4, 5, 2, 3, 1]   
1  potential||The potential couldn't be measured.  [5, 3, 4, 4, 3]   
2  potential||

In [ ]:

# Append synthetic supervision to strengthen rare-sense coverage.
train_df = pd.concat([train_df, syn_df])
print("TRAIN:", len(train_df))

TRAIN: 3072


In [ ]:
# Keep 20% of the mix as a validation set for Trainer callbacks.
train_df, val_df = train_test_split(
    train_df, test_size=0.20, random_state=42
)

print("TRAIN:", len(train_df), "VAL:", len(val_df))


TRAIN: 2457 VAL: 615


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    """Tokenize the (story, definition) pair and attach regression labels."""
    enc = tokenizer(
        batch["context"],
        batch["meaning"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    enc["labels"] = batch["label"]
    return enc


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Materialize Hugging Face datasets once to avoid recomputing tokens.
train_dataset = Dataset.from_pandas(train_df, preserve_index=False).map(tokenize_batch, batched=True)
val_dataset   = Dataset.from_pandas(val_df, preserve_index=False).map(tokenize_batch, batched=True)
dev_dataset   = Dataset.from_pandas(dev_df, preserve_index=False).map(tokenize_batch, batched=True)

Map:   0%|          | 0/2457 [00:00<?, ? examples/s]

Map:   0%|          | 0/615 [00:00<?, ? examples/s]

Map:   0%|          | 0/588 [00:00<?, ? examples/s]

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1,
    problem_type="regression"
)

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    """Track Spearman (ranking quality) and MAE (absolute error)."""
    preds, labels = eval_pred
    preds = preds.reshape(-1)

    sp = spearmanr(labels, preds).correlation
    mae = np.mean(np.abs(labels - preds))

    return {"spearman": float(sp), "mae": float(mae)}


In [ ]:
# DeBERTa-large finetuning setup that fits comfortably on a Colab GPU.
 args = TrainingArguments(
    output_dir="./tmp_deberta_regression",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)


In [16]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-3517603358.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Spearman,Mae
1,1.312800,1.137635,0.410090,0.889729
2,0.788300,0.936567,0.635107,0.744595
3,0.520100,0.764535,0.675281,0.665886
4,0.336000,0.745632,0.687502,0.646920
5,0.250000,0.731655,0.702501,0.650550
6,0.178400,0.779529,0.701392,0.660222
7,0.110900,0.805003,0.696618,0.689995
8,0.095100,0.714749,0.701230,0.641983
9,0.068800,0.708533,0.700396,0.640737
10,0.054400,0.740617,0.700923,0.658683


TrainOutput(global_step=1540, training_loss=0.440579084142462, metrics={'train_runtime': 319.4472, 'train_samples_per_second': 76.914, 'train_steps_per_second': 4.821, 'total_flos': 1.144881540601344e+16, 'train_loss': 0.440579084142462, 'epoch': 10.0})

In [18]:
preds = trainer.predict(dev_dataset).predictions.reshape(-1)
dev_labels = dev_df["label"].to_numpy()

global_spearman = spearmanr(dev_labels, preds).correlation
mae = np.mean(np.abs(dev_labels - preds))

print(f"\n=== DEV EVALUATION ===")
print(f"Spearman: {global_spearman:.4f}")
print(f"MAE:      {mae:.4f}")


=== DEV EVALUATION ===
Spearman: 0.5949
MAE:      0.7918


In [ ]:
def bootstrap_test_metrics(
    y_true,
    y_pred,
    y_stdev,
    groups,
    n_bootstrap=1000,
    seed=42
):
    """Bootstrap confidence intervals to understand metric variance."""
    rng = np.random.default_rng(seed)
    n = len(y_true)

    global_sps = []
    macro_sps = []
    maes = []
    acc_stdevs = []

    group_to_indices = defaultdict(list)
    for i, gid in enumerate(groups):
        group_to_indices[gid].append(i)

    for _ in range(n_bootstrap):
        # Resample indices with replacement to mimic new dev draws.
        idx = rng.integers(0, n, size=n)

        bt_true = y_true[idx]
        bt_pred = y_pred[idx]
        bt_stdev = y_stdev[idx]
        bt_groups = [groups[i] for i in idx]

        # ---- Global Spearman ----
        rho = spearmanr(bt_true, bt_pred).correlation
        global_sps.append(rho)

        # ---- MAE ----
        maes.append(np.mean(np.abs(bt_true - bt_pred)))

        # ---- Accuracy within stdev ----
        acc_stdevs.append(np.mean(np.abs(bt_true - bt_pred) <= bt_stdev))

        # ---- Macro Spearman ----
        local_group_map = defaultdict(list)
        for i, g in enumerate(bt_groups):
            local_group_map[g].append(i)

        group_corrs = []
        for g, idxs in local_group_map.items():
            gt = bt_true[idxs]
            gp = bt_pred[idxs]

            if np.all(gt == gt[0]):
                continue

            c = spearmanr(gt, gp).correlation
            if not np.isnan(c):
                group_corrs.append(c)

        if group_corrs:
            macro_sps.append(np.mean(group_corrs))
        else:
            macro_sps.append(np.nan)

    def summarize(arr):
        # Emit per-metric means plus 95% percentile-based bounds.
        arr = np.array(arr, dtype=float)
        return {
            "mean": float(np.nanmean(arr)),
            "ci_low": float(np.nanpercentile(arr, 2.5)),
            "ci_high": float(np.nanpercentile(arr, 97.5)),
        }

    return {
        "global_spearman": summarize(global_sps),
        "macro_spearman": summarize(macro_sps),
        "mae": summarize(maes),
        "acc_within_stdev": summarize(acc_stdevs),
    }


In [20]:
print("\nBOOTSTRAP RESULTS (TEST SET)")

bootstrap_results = bootstrap_test_metrics(
    y_true=dev_df["label"].to_numpy(),
    y_pred=preds,
    y_stdev=dev_df["stdev"].to_numpy(),
    groups=dev_df["group_id"].tolist(),
    n_bootstrap=1000
)


for metric, stats in bootstrap_results.items():
    print(
        f"{metric:20s}: "
        f"{stats['mean']:.4f} "
        f"[{stats['ci_low']:.4f}, {stats['ci_high']:.4f}]"
    )



BOOTSTRAP RESULTS (TEST SET)


/tmp/ipython-input-3268535183.py:52: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  c = spearmanr(gt, gp).correlation


global_spearman     : 0.5944 [0.5332, 0.6502]
macro_spearman      : 0.5643 [0.4746, 0.6507]
mae                 : 0.7923 [0.7407, 0.8416]
acc_within_stdev    : 0.6156 [0.5782, 0.6548]
